In [20]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader 
import transformers
from transformers import GPT2Tokenizer, GPT2Model
from tqdm import tqdm

In [2]:
data = pd.read_feather('../mini_codenet/data/split/reward_val.ftr')
data.head()

,level_0,index,submission_id,problem_id,language,filename_ext,status,cpu_time,memory,code_size,accuracy,solution,problem_statement
0,761513,7854,s355775556,p03053,C++,cpp,Runtime Error,342.0,5120.0,1831,None,#include <stdio.h>\n#include <stdlib.h>\n#incl...,Score : 300 points \n Problem Statement You ar...
1,761514,15586,s361808912,p03053,Python,py,Runtime Error,1062.0,150772.0,692,None,"H, W = map(int, input().split())\nA = []\nfor ...",Score : 300 points \n Problem Statement You ar...
2,761515,14291,s920384889,p03053,C++,cpp,Runtime Error,389.0,512.0,1372,None,#include <iostream>\nusing namespace std;\n\ni...,Score : 300 points \n Problem Statement You ar...
3,761516,14222,s539766324,p03053,Python,py,Runtime Error,1064.0,138392.0,638,None,# coding:utf-8\n\nif __name__=='__main__':\n ...,Score : 300 points \n Problem Statement You ar...
4,761517,1227,s849699734,p03053,Python,py,Runtime Error,758.0,308244.0,13571,None,"import sys,bisect,string,math,time,functools,r...",Score : 300 points \n Problem Statement You ar...


In [3]:
data.iloc[0]["solution"]

'#include <stdio.h>\n#include <stdlib.h>\n#include <math.h>\n#include <string.h>\n#include <queue>\nusing namespace std;\ntypedef unsigned int uint;\ntypedef long long ll;\ntypedef pair<int, int> P;\nconst int c_INF = 111111111;\nconst int c_mINF = -111111111;\nconst int c_YET = -1;\nconst int c_Dx[4] = { 0,0,1,-1 };\nconst int c_Dy[4] = { 1,-1,0,0 };\n\nvoid debug();\n\n\nint H, W, dst[1000][1000];\nchar  grid[1000][1000];\nvoid BFS(int sy, int sx);\nint main() {\n\tscanf("%d%d", &H, &W);\n\tfor (int i = 0; i < H; i++)scanf("%s", grid[i]);\n\n\tfor (int i = 0; i < H; i++)\n\t\tfor (int j = 0; j < W; j++)dst[i][j] = c_YET;\n\n\tfor (int i = 0; i < H; i++)\n\t\tfor (int j = 0; j < W; j++)\n\t\t\tif (grid[i][j] == \'#\')BFS(i, j);\n\n\tint max = c_mINF;\n\tfor (int i = 0; i < H; i++)\n\t\tfor (int j = 0; j < W; j++)\n\t\t\tif (dst[i][j] > max)max = dst[i][j];\n\n\tprintf("%d", max);\n\n}\n\n\nvoid BFS(int sy,int sx) {\n\tdst[sy][sx] = 0;\n\tqueue<P> que;\n\tque.push(P(sy, sx));\n\n\twhil

In [39]:
class CodeNetDataset(Dataset):
    def __init__(self, feather_path, tokenizer):
        self.data = pd.read_feather(feather_path)
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instance = self.data.iloc[idx]
        # language = self.tokenizer(instance["language"], return_tensors='pt')
        problem = self.tokenizer(instance["language"] + "::" + instance["problem_statement"], max_length=1028, padding='max_length', truncation=True, return_tensors='pt')
        code_solution = self.tokenizer(instance["solution"], max_length=1028, padding='max_length', truncation=True, return_tensors='pt')

        return problem, code_solution

In [40]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2Model.from_pretrained('gpt2-large')
cnds = CodeNetDataset('../mini_codenet/data/split/evaluate_test.ftr', tokenizer)

Using pad_token, but it is not set yet.


In [41]:
p, s = cnds[0]
print(p.input_ids.shape)

torch.Size([1, 1028])


In [21]:
# get the max len of all tokenizations
# this cell takes around 20 minutes
maxlen_i = 0
maxlen_j = 0
for i, j in tqdm(cnds):
    bi, wi = i.input_ids.shape
    bj, wj = j.input_ids.shape

    if wi > maxlen_i:
        maxlen_i = wi

    if wj > maxlen_j:
        maxlen_j = wj

print(maxlen_i)
print(maxlen_j)

100%|██████████| 190378/190378 [18:58<00:00, 167.22it/s]  

1465
293127


In [51]:

dl = DataLoader(cnds, batch_size=10, shuffle=True)
for i, batch in enumerate(dl):
    print(i, batch[1].input_ids.shape)
    break

0 torch.Size([10, 1, 1028])
